In [1]:
import matplotlib.pyplot as plt
import numpy as np
import random as rnd
import math as m
import pandas as pd
import scipy.stats as stat
import scipy.optimize as opt
import seaborn as sb
from scipy.integrate import quad
from scipy.integrate import dblquad
from scipy.special import comb
from sklearn.linear_model import LinearRegression
plt.style.use('ggplot')

my_norm = lambda exp, disp: stat.norm(loc = exp, scale = m.sqrt(disp))
from mpmath import mp
#from mpmath import e

In [2]:
#import io
f = open("data.txt", mode="r", encoding="utf-8")

##### Сначала разбиваем на серии

In [3]:
def get_num(sym):
    sym = sym.lower()
    if sym == 'к':
        return 0;
    if sym == 'н':
        return 1;
    if sym == 'б':
        return 2

In [4]:
counter = 0;
series = []
with open('data.txt', mode = 'r', encoding = 'UTF-8') as f:
    for line in f:
        if (len(line) > 6):
            series.append([])
        else:
            series[-1].append((get_num(line[0]), get_num(line[2])))
        counter += 1

##### Далее получаем матрицу переходов

In [5]:
P1 = np.zeros((9,3))

In [6]:
P1 = np.zeros((9,3))
for ser in series:
    for i in range(1, len(ser)):
        P1[ser[i-1][0] * 3 + ser[i-1][1]][ser[i][0]] += 1;
        P1[ser[i-1][0] + ser[i-1][1] * 3][ser[i][1]] += 1;
        
P1

array([[ 7., 14., 11.],
       [11., 15., 23.],
       [ 9., 15., 10.],
       [14., 14., 21.],
       [12.,  8., 10.],
       [22.,  9., 14.],
       [13., 13.,  8.],
       [11., 18., 16.],
       [17., 18., 15.]])

In [7]:
P = np.copy(P1)
P = P.astype(float)
for i in range(9):
    P[i] = P[i] / np.sum(P[i])

#P = np.vstack((P, [1/3,1/3,1/3]))
np.round(P,2)

array([[0.22, 0.44, 0.34],
       [0.22, 0.31, 0.47],
       [0.26, 0.44, 0.29],
       [0.29, 0.29, 0.43],
       [0.4 , 0.27, 0.33],
       [0.49, 0.2 , 0.31],
       [0.38, 0.38, 0.24],
       [0.24, 0.4 , 0.36],
       [0.34, 0.36, 0.3 ]])

In [8]:
def ideal_player(prev_turn):
    opponent_exp = list(stat.multinomial(1, P[prev_turn[0] + 3 * prev_turn[1], :]).rvs()[0]).index(1)
    return (opponent_exp - 1) % 3
def my_player(prev_turn):
    opponent_exp = list(P[prev_turn[0] + 3 * prev_turn[1], :]).index(np.max(P[prev_turn[0] + 3 * prev_turn[1], :]))
    return (opponent_exp - 1) % 3

##### Он и правда обыгрывает людей на нашей выборке
Причем как "первого", так и "второго".

In [9]:
wins = 0; loses = 0;
for i in range(500):
    for ser in series:
        for i in range(1, len(ser)):
            if (my_player(ser[i-1]) - ser[i][1]) % 3 == 1:
                loses += 1
            if (my_player(ser[i-1]) - ser[i][1]) % 3 == 2:
                wins += 1

                
print(wins/500);print(loses/500)

81.0
55.0


In [10]:
wins = 0; loses = 0; n = 500
for i in range(n):
    for ser in series:
        for i in range(1, len(ser)):
            if (my_player(ser[i-1][::-1]) - ser[i][0]) % 3 == 1:
                loses += 1
            if (my_player(ser[i-1][::-1]) - ser[i][0]) % 3 == 2:
                wins += 1

                
print(wins/n);print(loses/n)

75.0
52.0


In [11]:
wins = 0; loses = 0;
for i in range(500):
    for ser in series:
        for i in range(1, len(ser)):
            if (ideal_player(ser[i-1]) - ser[i][1]) % 3 == 1:
                loses += 1
            if (ideal_player(ser[i-1]) - ser[i][1]) % 3 == 2:
                wins += 1

                
print(wins/500);print(loses/500)

63.996
58.976


In [12]:
wins = 0; loses = 0; n = 500
for i in range(n):
    for ser in series:
        for i in range(1, len(ser)):
            if (ideal_player(ser[i-1][::-1]) - ser[i][0]) % 3 == 1:
                loses += 1
            if (ideal_player(ser[i-1][::-1]) - ser[i][0]) % 3 == 2:
                wins += 1

                
print(wins/n);print(loses/n)

65.428
59.632


##### Бот на основе такой ЦМ

In [13]:
def get_letter(num):
    if num == 0:
        return 'камень';
    if num == 1:
        return 'ножницы';
    if num == 2:
        return 'бумагу'

In [14]:
print('Введите к, н или б'); wins = 0; loses = 0;games = 0;

bot_turn = list(stat.multinomial(1, [1/3,1/3,1/3]).rvs()[0]).index(1)
A = get_num(input().split()[0])
print("Бот показал", get_letter(bot_turn), end = '; ')

if ((A - bot_turn) % 3 == 1):
    loses += 1;
    print("Вы проиграли")

if ((A - bot_turn) % 3 == 2):
    wins += 1;
    print("Вы выиграли!")
    
if ((A - bot_turn) % 3 == 0):
    print("Ничья!")

pr_turn = (bot_turn, A)
print('Введите к, н или б')
while games < 41:
    bot_turn = ideal_player(pr_turn)
    A =  input().split()
    
    if (len(A) > 1 or not A):
        print('Неверный ввод, попробуйте еще раз')
        continue
    if not (A[0] =='к' or A[0] == 'н' or A[0] == 'б'):
        print('Неверный ввод, попробуйте еще раз')
        continue
        
    A = get_num(A[0])
    
    print("Бот показал", get_letter(bot_turn), end = '; ')
    if ((A - bot_turn) % 3 == 1):
        loses += 1;
        print("Вы проиграли")

    if ((A - bot_turn) % 3 == 2):
        wins += 1;
        print("Вы выиграли!")

    if ((A - bot_turn) % 3 == 0):
        print("Ничья!")

    pr_turn = (bot_turn, A)
    games += 1
    
print('Побед:', wins, 'Поражений:', loses)

Введите к, н или б
к
Бот показал бумагу; Вы проиграли
Введите к, н или б
к
Бот показал камень; Ничья!
к
Бот показал камень; Ничья!
к
Бот показал бумагу; Вы проиграли

Неверный ввод, попробуйте еще раз
к
Бот показал камень; Ничья!
к
Бот показал камень; Ничья!
к
Бот показал ножницы; Вы выиграли!
к
Бот показал ножницы; Вы выиграли!
к
Бот показал бумагу; Вы проиграли
к
Бот показал камень; Ничья!

Неверный ввод, попробуйте еще раз
к
Бот показал ножницы; Вы выиграли!
к
Бот показал ножницы; Вы выиграли!
к
Бот показал бумагу; Вы проиграли
к
Бот показал бумагу; Вы проиграли
к
Бот показал ножницы; Вы выиграли!
к
Бот показал ножницы; Вы выиграли!
кк
Неверный ввод, попробуйте еще раз
к
Бот показал бумагу; Вы проиграли
к
Бот показал бумагу; Вы проиграли
к
Бот показал бумагу; Вы проиграли
к
Бот показал ножницы; Вы выиграли!
к
Бот показал ножницы; Вы выиграли!
к
Бот показал камень; Ничья!
к
Бот показал камень; Ничья!
к
Бот показал ножницы; Вы выиграли!
к
Бот показал ножницы; Вы выиграли!
к
Бот показа

##### Менее словоохотливая версия бота:
С помощью нее можно было бы дополнить нашу выборку

In [16]:
print('Введите к, н или б'); wins = 0; loses = 0;games = 0;

bot_turn = list(stat.multinomial(1, [1/3,1/3,1/3]).rvs()[0]).index(1)
A = get_num(input().split()[0])
print(get_letter(bot_turn)[0]+'\n')

if ((A - bot_turn) % 3 == 1):
    loses += 1;
    #print("Вы проиграли")

if ((A - bot_turn) % 3 == 2):
    wins += 1;
    #print("Вы выиграли!")
    

pr_turn = (bot_turn, A)
while games < 41:
    bot_turn = ideal_player(pr_turn)
    A =  input().split()
    
    if (len(A) > 1 or not A):
        print('Неверный ввод, попробуйте еще раз')
        continue
    if not (A[0] =='к' or A[0] == 'н' or A[0] == 'б'):
        print('Неверный ввод, попробуйте еще раз')
        continue
        
    A = get_num(A[0])
    
    print( get_letter(bot_turn)[0]+'\n')
    if ((A - bot_turn) % 3 == 1):
        loses += 1;
        #print("Вы проиграли")

    if ((A - bot_turn) % 3 == 2):
        wins += 1;
        #print("Вы выиграли!")

    pr_turn = (bot_turn, A)
    games += 1
    
print('Побед:', wins, 'Поражений:', loses)

Введите к, н или б
к
б

к
к

к
к

к
б

к
к


Неверный ввод, попробуйте еще раз
к
к

к
к

к
н

к
к

к
н

к
к

к
к

к
к

к
к

к
н

к
б

к
к

к
н

к
н

к
к

к
н

к
к

к
н

к
к

к
к

к
к

к
н

к
н

к
н

к
н

к
н

к
б

к
к

к
к

к
б

к
к

к
к

к
к

к
к

к
к

к
б

к
н

Побед: 13 Поражений: 6


У меня получилось 12 побед на 14 поражений.

У Идалии: 17-14, 13-20, 14-13, 15-9, 11-15. Сумма 70 - 71.

###### Это детерминированный бот, основанный на нашей выборке

In [ ]:
print('Введите к, н или б'); wins = 0; loses = 0;games = 0;

bot_turn = list(stat.multinomial(1, [1/3,1/3,1/3]).rvs()[0]).index(1)
A = get_num(input().split()[0])
print("Бот показал", get_letter(bot_turn), end = '; ')

if ((A - bot_turn) % 3 == 1):
    loses += 1;
    print("Вы проиграли")

if ((A - bot_turn) % 3 == 2):
    wins += 1;
    print("Вы выиграли!")
    
if ((A - bot_turn) % 3 == 0):
    print("Ничья!")

pr_turn = (bot_turn, A)
print('Введите к, н или б')
while games < 41:
    bot_turn = my_player(pr_turn)
    A =  input().split()
    
    if (len(A) > 1 or not A):
        print('Неверный ввод, попробуйте еще раз')
        continue
    if not (A[0] =='к' or A[0] == 'н' or A[0] == 'б'):
        print('Неверный ввод, попробуйте еще раз')
        continue
        
    A = get_num(A[0])
    
    print("Бот показал", get_letter(bot_turn), end = '; ')
    if ((A - bot_turn) % 3 == 1):
        loses += 1;
        print("Вы проиграли")

    if ((A - bot_turn) % 3 == 2):
        wins += 1;
        print("Вы выиграли!")

    if ((A - bot_turn) % 3 == 0):
        print("Ничья!")

    pr_turn = (bot_turn, A)
    games += 1
    
print('Побед:', wins, 'Поражений:', loses)

In [ ]:
#------------------------------------------------------------------------------------------

https://lifehacker.ru/kak-vsegda-vyigryvat-v-kamen-nozhnicy-bumaga/ рекомедует детерминированную стратегию:

Все еще не до конца понятно? Вот какие выигрышные стратегии помогут вам оставаться непобедимыми:

###### Если вы победили в последней игре…
…выкинув камень, переходите к ножницам в следующей схватке

…выкинув ножницы, переходите к бумаге в следующей схватке

…выкинув бумагу, переходите к камню в следующей схватке
###### Если вы проиграли в последней игре (и противник не в курсе этой методики)…
…выкинув камень, переходите к ножницам в следующей схватке

…выкинув ножницы, переходите к бумаге в следующей схватке

…выкинув бумагу, переходите к камню в следующей схватке

In [17]:
def lihack(prev_turn):
    if (prev_turn[0] == prev_turn[1]):
        return list(stat.multinomial(1, [1/3, 1/3, 1/3]).rvs()[0]).index(1)
    return (prev_turn[0] + 1) %3

In [18]:
wins = 0; loses = 0;
n = 500
for i in range(n):
    for ser in series:
        for i in range(1, len(ser)):
            if (lihack(ser[i-1]) - ser[i][1]) % 3 == 1:
                loses += 1
            if (lihack(ser[i-1]) - ser[i][1]) % 3 == 2:
                wins += 1
                
print(wins/n);print(loses/n)

59.704
60.678


In [19]:
wins = 0; loses = 0;
n = 500
for i in range(n):
    for ser in series:
        for i in range(1, len(ser)):
            if (lihack(ser[i-1][::-1]) - ser[i][0]) % 3 == 1:
                loses += 1
            if (lihack(ser[i-1][::-1]) - ser[i][0]) % 3 == 2:
                wins += 1
                
print(wins/n);print(loses/n)

52.572
60.7


Метод не работает

##### Бот на основе "слабого" алгоритма, описанного выше

In [ ]:
print('Введите к, н или б'); wins = 0; loses = 0;games = 0;

bot_turn = list(stat.multinomial(1, [1/3,1/3,1/3]).rvs()[0]).index(1)
A = get_num(input().split()[0])
print("Бот показал", get_letter(bot_turn), end = '; ')

if ((A - bot_turn) % 3 == 1):
    loses += 1;
    print("Вы проиграли")

if ((A - bot_turn) % 3 == 2):
    wins += 1;
    print("Вы выиграли!")
    
if ((A - bot_turn) % 3 == 0):
    print("Ничья!")

pr_turn = (bot_turn, A)
print('Введите к, н или б')
while games < 41:
    bot_turn = lihack(pr_turn)
    A =  input().split()
    
    if (len(A) > 1 or not A):
        print('Неверный ввод, попробуйте еще раз')
        continue
    if not (A[0] =='к' or A[0] == 'н' or A[0] == 'б'):
        print('Неверный ввод, попробуйте еще раз')
        continue
        
    A = get_num(A[0])
    
    print("Бот показал", get_letter(bot_turn), end = '; ')
    if ((A - bot_turn) % 3 == 1):
        loses += 1;
        print("Вы проиграли")

    if ((A - bot_turn) % 3 == 2):
        wins += 1;
        print("Вы выиграли!")

    if ((A - bot_turn) % 3 == 0):
        print("Ничья!")

    pr_turn = (bot_turn, A)
    games += 1
    
print('Побед:', wins, 'Поражений:', loses)